# 5. Объединение DataFrame: знакомимся с новыми данными
✍ На практике источники данных редко ограничиваются одной таблицей. Например, если мы работаем с базой данных, то данные в ней могут быть представлены в виде нескольких десятков таблиц, каждая из которых несёт отдельную информацию. Если вы делаете выгрузку из базы напрямую, не объединяя таблицы в единую структуру средствами SQL, вам необходимо знать, как работать с такими таблицами средствами Pandas. 

## С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ?

В этой части модуля мы будем работать с популярным датасетом [MovieLens](https://grouplens.org/datasets/movielens/), в котором собраны логи некоторой рекомендательной системы фильмов.

Скачать таблицы, с которыми мы будем работать, можно [здесь](https://lms.skillfactory.ru/assets/courseware/v1/bcd86def807792174f50d955515c8be0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/movies_data.zip) (файлы находятся в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

Наши данные представляют собой четыре таблицы:
- ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.

In [1]:
import pandas as pd

ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')
ratings1.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


Назначение столбцов

1. userId — уникальный идентификатор пользователя, который выставил оценку;
2. movieId — уникальный идентификатор фильма;
3. rating — рейтинг фильма.

- dates — таблица с датами выставления всех оценок.

In [2]:
dates = pd.read_csv('data/dates.csv', sep=',')
dates.head()

,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


1. date — дата и время выставления оценки фильму.

- movies — таблица с информацией о фильмах:

In [3]:
movies = pd.read_csv('data/movies.csv', sep=',')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


1. movieId — уникальный идентификатор фильма;
2. title — название фильма и год его выхода;
3. genres — жанры фильма.

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

1. Склеим таблицы ratings1 и ratings2 в единую структуру. Термин «склеить» в данном случае обозначает конкатенацию — присоединение одной таблицы к другой.
2. К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.
3. Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

## ЗАЧЕМ ХРАНИТЬ ДАННЫЕ В РАЗНЫХ ТАБЛИЦАХ?

Конечно, здорово, если все необходимые данные лежат в одной таблице, но на практике такое случается редко по двум объективным причинам:

Часто данные формируются несколькими независимыми процессами, каждый из которых хранит данные в своей таблице.

Например, данные для отчёта по продажам могут состоять из списка банковских транзакций, курсов валют от Центробанка и планов отдела продаж из внутренней CRM. Все эти три таблицы, скорее всего, будут формироваться независимыми друг от друга системами. Объединять их в один отчёт придётся вам.

Хранить все данные в одной таблице часто очень накладно для ёмкости диска.

Например, названия фильмов в наших данных хранятся в отдельной небольшой таблице. А в логах, которые могут растягиваться на многие миллионы строк, вместо названия фильма стоит его идентификатор. Числовой идентификатор фильма занимает на диске гораздо меньше места, чем длинное название, поэтому логи с идентификаторами будут занимать гораздо меньше места, чем единая таблица с названиями.

✍ Прежде чем приступать к объединению таблиц, предлагаем вам исследовать информацию, которая в них содержится ↓

### Задание 5.1
Значения из какого столбца таблиц ratings1 и ratings2 можно расшифровать с помощью таблицы movies?
- userId
- movieId
- title
- genres
- rating

Ответ: movie_id

### Задание 5.2
Сколько уникальных фильмов представлено в таблице movies?

In [4]:
movies.shape[0]

9742

### Задание 5.3
Сколько уникальных пользователей в таблице ratings1?

In [6]:
ratings1['userId'].nunique()

274

### Задание 5.4
В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу dates.

In [10]:
dates['date'] = pd.to_datetime(dates['date'])
dates['year'] = dates['date'].dt.year
dates['year'].value_counts().keys()[0]

2000